# ABS Quarterly Job Vacancies 6354

## Python set-up

In [1]:
# system imports
import sys

# analytic imports
import pandas as pd
import matplotlib.pyplot as plt

# local imports
from abs_data_capture import (
    get_fs_constants,
    get_meta_constants,
    get_ABS_meta_and_data,
    get_plot_constants,
    clear_cache,
    find_id,
)
from plotting import (
    recalibrate,
    set_chart_dir,
    clear_chart_dir,
    seas_trend_plot,
    plot_covid_recovery,
)

# pandas display settings
pd.options.display.max_rows = None
pd.options.display.max_columns = 999
pd.options.display.max_colwidth = 100

## Get data from ABS

In [2]:
if False:
    clear_cache()

In [3]:
catalogue_id = "6354"
source, CHART_DIR, META_DATA = get_fs_constants(catalogue_id)

abs_dict = get_ABS_meta_and_data(catalogue_id)
if abs_dict is None:
    sys.exit(-1)

meta = abs_dict[META_DATA]
RECENT, plot_times, plot_tags = get_plot_constants(meta)
did_col, id_col, table_col, type_col, unit_col = get_meta_constants()

Found URL for a ZIP file on ABS web page
Retrieving zip-file from cache ...
Extracting DataFrames from the zip-file ...


## Plot

### Set up

In [4]:
clear_chart_dir(CHART_DIR)
set_chart_dir(CHART_DIR)

plt.style.use("fivethirtyeight")

# display charts
SHOW = False

### Headline charts

In [5]:
def headline_plots() -> None:
    did = "Job Vacancies ;  Australia ;"
    table = "1"
    plot_data = pd.DataFrame()
    for service_type in ("Seasonally Adjusted", "Trend"):
        selector = {
            service_type: type_col,
            table: table_col,
            did: did_col,
        }
        series_id, units = find_id(meta, selector, verbose=False)
        plot_data[service_type] = abs_dict[table][series_id]

    plot_data, units = recalibrate(plot_data, units)
    plot_data = plot_data.resample(rule=plot_data.index.freq).mean()
    title = did.replace(' ;  ', ': ').replace(' ;', '')
    seas_trend_plot(
        plot_data,
        title=title,
        ylabel=f"{units} / Quarter",
        dropna=False,
        rfooter=f'{source} T{table}',
        show=SHOW,
    )

    plot_covid_recovery(
        plot_data["Seasonally Adjusted"],
        title=title,
        ylabel=f"{units} / Quarter",
        tags="COVID",
        rfooter=f'{source} T{table}',
        show=SHOW,
    )

In [6]:
headline_plots()

### Industry sector charts

In [7]:
def sectors() -> None:
    table = '4'
    rows = meta[
        (meta[table_col] == table)
        & meta[did_col].str.contains("^Job Vacancies", regex=True)
    ]

    for index, row in rows.iterrows():
        series_id, units = row[id_col], row[unit_col]
        did, series_type = row[did_col], row[type_col]
        series, units = recalibrate(abs_dict[table][series_id], units)
        series.name = f'{series_type} series'
    
        plot_covid_recovery(
            series,
            title=did.replace(' ;  ', ': ').replace(' ;', ''),
            ylabel=f"{units} / Quarter",
            rfooter=f'{source} T{table}',
            lfooter=f'Australia. ',
            show=SHOW,
        )

In [8]:
sectors()

### State charts

In [9]:
def states() -> None:
    table = '1'
    series_type = 'Original'
    rows = meta[
        (meta[table_col] == table)
        & meta[did_col].str.contains("^Job Vacancies", regex=True)
        & meta[type_col].str.contains(series_type, regex=False)
    ]

    for index, row in rows.iterrows():
        series_id, units = row[id_col], row[unit_col]
        did = row[did_col]
        series, units = recalibrate(abs_dict[table][series_id], units)
        series.name = f'{series_type} series'
    
        plot_covid_recovery(
            series,
            title=did.replace(' ;  ', ': ').replace(' ;', ''),
            ylabel=f"{units} / Quarter",
            rfooter=f'{source} T{table}',
            show=SHOW,
        )

In [10]:
states()

## Finished

In [11]:
# watermark
%load_ext watermark
%watermark -u -n -t -v -iv -w

Last updated: Sat Apr 01 2023 08:49:43

Python implementation: CPython
Python version       : 3.11.2
IPython version      : 8.12.0

matplotlib: 3.7.1
pandas    : 1.5.3
sys       : 3.11.2 | packaged by conda-forge | (main, Mar 31 2023, 17:54:27) [Clang 14.0.6 ]

Watermark: 2.3.1



In [12]:
print("Finished")

Finished
